In [1]:
import os
os.environ["SPARK_HOME"] = "D:\RISET\Spark\spark-3.1.1-bin-hadoop2.7"
from platform import python_version
print(python_version())

3.7.10


In [2]:
import numpy as np
import pandas as pd
import xlwings as xw
import folium
import docplex.mp.solution as mp_sol
from docplex.mp.model import Model
from geopy.distance import great_circle
import json

n = 15
customer = [i for i in range(1, n + 1)]
customer2 = [i for i in range(1, n + 1)]
node = [0] + customer
arcos = [(i,j) for i in node for j in node if i != j]
f = open('pelanggan3.json')
data = json.load(f)

# get customer demand and location data
df = pd.DataFrame(data)

# get distance & duration data
distance = np.loadtxt('distance.txt')
duration = np.loadtxt('duration.txt')
vehicle_capacity = 250

# Start time
e = [df['ready_time'][i] for i in range(n + 1)]
e[0] = 0

# Due time
l = [df['due_time'][i] for i in range(n + 1)]
 

# Service time
ser = [df['service_time'][i] for i in range(n + 1)]
ser[0] = 0

K = 10000
df

,name,latitude,longitude,demand,ready_time,due_time,id,service_time,best_arrive
0,Depo-Gas Station - Pertamina - 34-15316,-6.293370,106.648566,0,0,5000,0,NaN,NaN
1,Customer 1,-6.308553,106.653698,46,304,1504,1,300.0,0.0
2,Customer 2,-6.306463,106.653170,26,177,1377,2,300.0,0.0
3,Customer 3,-6.313126,106.652578,26,69,1269,3,300.0,0.0
4,Customer 4,-6.314943,106.652226,27,107,1307,4,300.0,0.0
5,Customer 5,-6.312494,106.646621,16,3,1203,5,300.0,0.0
6,Customer 6,-6.301097,106.642021,56,391,1591,6,300.0,0.0
7,Customer 7,-6.303528,106.641497,26,83,1283,7,300.0,0.0
8,Customer 8,-6.314158,106.652149,30,387,1587,8,300.0,0.0
9,Customer 9,-6.309986,106.653968,56,38,1238,9,300.0,0.0


In [3]:
mdl = Model('CVRPTW')
x = mdl.binary_var_dict(arcos, name='x')
u = mdl.continuous_var_dict(customer, name='u')
t = mdl.continuous_var_dict(customer2, name='t')

In [4]:
mdl.minimize(mdl.sum(distance[(i, j)] * x[(i, j)] for i, j in arcos))
mdl.add_constraints(mdl.sum(x[(i,j)] for i,j in arcos if i==h)== 1 for h in customer)
mdl.add_constraints(mdl.sum(x[(i,j)] for i,j in arcos if j==h)== 1 for h in customer)
for i, j in arcos:
    if i!=0 and j!=0:
        mdl.add_indicator(x[(i,j)], t[i] + ser[i] + duration[i][j] <= t[j])
mdl.add_constraints(e[i] <= t[i] for i in customer)
mdl.add_constraints(t[i] <= l[i] for i in customer)

for i, j in arcos:
    if i!=0 and j!=0:
        mdl.add_indicator(x[(i,j)], u[i] + df.demand[j] == u[j])
mdl.add_constraints(df.demand[i] <= u[i] for i in customer)
mdl.add_constraints(u[i] <= vehicle_capacity for i in customer)


[docplex.mp.LinearConstraint[](u_1,LE,250),
 docplex.mp.LinearConstraint[](u_2,LE,250),
 docplex.mp.LinearConstraint[](u_3,LE,250),
 docplex.mp.LinearConstraint[](u_4,LE,250),
 docplex.mp.LinearConstraint[](u_5,LE,250),
 docplex.mp.LinearConstraint[](u_6,LE,250),
 docplex.mp.LinearConstraint[](u_7,LE,250),
 docplex.mp.LinearConstraint[](u_8,LE,250),
 docplex.mp.LinearConstraint[](u_9,LE,250),
 docplex.mp.LinearConstraint[](u_10,LE,250),
 docplex.mp.LinearConstraint[](u_11,LE,250),
 docplex.mp.LinearConstraint[](u_12,LE,250),
 docplex.mp.LinearConstraint[](u_13,LE,250),
 docplex.mp.LinearConstraint[](u_14,LE,250),
 docplex.mp.LinearConstraint[](u_15,LE,250)]

In [5]:
mdl.parameters.timelimit = 60
# mdl.parameters.parallel = 1
# mdl.parameters.threads =  0
# mdl.parameters.benders.strategy = -1
# mdl.parameters.mip.limits.cutsfactor = 0
# mdl.parameters.mip.tolerances.mipgap = 0.1
# mdl.parameters.mip.strategy.branch = 0

###Menghitung Waktu
import time
start_timer = time.time()

solucion = mdl.solve(log_output=True)

###Menghitung Waktu
done_timer = time.time()
elapsed = done_timer - start_timer

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               60
Found incumbent of value 84444.000000 after 0.00 sec. (0.02 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 30 rows and 0 columns.
MIP Presolve modified 105 coefficients.
Aggregator did 105 substitutions.
Reduced MIP has 135 rows, 360 columns, and 765 nonzeros.
Reduced MIP has 240 binaries, 0 generals, 0 SOSs, and 210 indicators.
Presolve time = 0.02 sec. (0.84 ticks)
Probing time = 0.00 sec. (1.09 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 135 rows, 360 columns, and 765 nonzeros.
Reduced MIP has 240 binaries, 0 generals, 0 SOSs, and 210 indicators.
Presolve time = 0.00 sec. (0.52 ticks)
Probing time = 0.00 sec. (1.05 ticks)
Clique table members: 135.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root 

In [6]:
mdl.get_solve_status()

<JobSolveStatus.FEASIBLE_SOLUTION: 1>

In [7]:
# Encontrar las rutas por separado
route = []
total_dis = []
for h in customer:
    if x[(0, h)].solution_value > 0.9:
        arcos_route = [(0, h)]
        item_dis = distance[0][h]
        j = h
        while j!=0:
            i = j
            for k in node:
                if i!=k and x[(i, k)].solution_value > 0.9:
                    j = k
                    arcos_route.append((i, j))
                    item_dis+= distance[i][j]
                    break
        route.append(arcos_route)
        print(arcos_route)
        print("----------------------")
        total_dis.append(item_dis)


[(0, 9), (9, 3), (3, 8), (8, 4), (4, 1), (1, 2), (2, 0)]
----------------------
[(0, 10), (10, 11), (11, 0)]
----------------------
[(0, 12), (12, 5), (5, 15), (15, 14), (14, 7), (7, 13), (13, 6), (6, 0)]
----------------------


In [8]:
best_distance = 0
for n in range(len(route)):
    print("Rute "+str(n))
    print(str(route[n])+" -> "+str(total_dis[n]))
    best_distance+=total_dis[n]
print("Total Distace : "+str(best_distance))
print(elapsed)

Rute 0
[(0, 9), (9, 3), (3, 8), (8, 4), (4, 1), (1, 2), (2, 0)] -> 6898.0
Rute 1
[(0, 10), (10, 11), (11, 0)] -> 3138.0
Rute 2
[(0, 12), (12, 5), (5, 15), (15, 14), (14, 7), (7, 13), (13, 6), (6, 0)] -> 10161.0
Total Distace : 20197.0
60.05540418624878


In [9]:
best_distance = 0
for n in range(len(route)):
    print("Rute "+str(n))
    print(str(route[n])+" -> "+str(total_dis[n]))
    s_time = 0
    for rt in route[n]:
        (i,j) = rt
        s_time += duration[i][j]
        print(str(rt)+" --> "+str(e[j])+" - "+str(l[j])+" --> "+str(s_time))
        s_time += ser[j]
    print("----------")
    best_distance+=total_dis[n]
print("Total Distace : "+str(best_distance))
print(elapsed)

Rute 0
[(0, 9), (9, 3), (3, 8), (8, 4), (4, 1), (1, 2), (2, 0)] -> 6898.0
(0, 9) --> 38 - 1238 --> 407.0
(9, 3) --> 69 - 1269 --> 806.0
(3, 8) --> 387 - 1587 --> 1151.0
(8, 4) --> 107 - 1307 --> 1503.0
(4, 1) --> 304 - 1504 --> 1899.0
(1, 2) --> 177 - 1377 --> 2234.0
(2, 0) --> 0 - 5000 --> 2870.0
----------
Rute 1
[(0, 10), (10, 11), (11, 0)] -> 3138.0
(0, 10) --> 255 - 1455 --> 266.0
(10, 11) --> 11 - 1211 --> 739.0
(11, 0) --> 0 - 5000 --> 1206.0
----------
Rute 2
[(0, 12), (12, 5), (5, 15), (15, 14), (14, 7), (7, 13), (13, 6), (6, 0)] -> 10161.0
(0, 12) --> 41 - 1241 --> 377.0
(12, 5) --> 3 - 1203 --> 944.0
(5, 15) --> 176 - 1376 --> 1373.0
(15, 14) --> 155 - 1355 --> 1716.0
(14, 7) --> 83 - 1283 --> 2199.0
(7, 13) --> 890 - 2090 --> 2501.0
(13, 6) --> 391 - 1591 --> 2923.0
(6, 0) --> 0 - 5000 --> 3598.0
----------
Total Distace : 20197.0
60.05540418624878
